# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 7 2022 11:34AM,248172,18,EMERSA-8622359,"Emersa Waterbox, LLC",Released
1,Oct 7 2022 11:27AM,248171,10,MSP214575,Methapharm-G,Released
2,Oct 7 2022 11:26AM,248169,10,CTF0011442,"Citieffe, Inc.",Released
3,Oct 7 2022 11:26AM,248170,10,MSP214573,Methapharm-G,Released
4,Oct 7 2022 11:24AM,248167,10,MSP214481,Methapharm-G,Released
5,Oct 7 2022 11:20AM,248166,10,SOTAH0000805,"Nextsource Biotechnology, LLC",Released
6,Oct 7 2022 10:58AM,248163,10,SONSB0001974,"Nextsource Biotechnology, LLC",Released
7,Oct 7 2022 10:53AM,248161,10,MSP214529(7062160),Methapharm-G,Released
8,Oct 7 2022 10:53AM,248161,10,MSP214571,Methapharm-G,Released
9,Oct 7 2022 10:53AM,248161,10,MSP214655,Methapharm-G,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
33,248167,Released,1
34,248169,Released,1
35,248170,Released,1
36,248171,Released,1
37,248172,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
248167,NaN,NaN,1.0
248169,NaN,NaN,1.0
248170,NaN,NaN,1.0
248171,NaN,NaN,1.0
248172,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248074,0.0,5.0,37.0
248085,0.0,0.0,1.0
248097,0.0,1.0,0.0
248115,0.0,3.0,6.0
248117,0.0,0.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248074,0,5,37
248085,0,0,1
248097,0,1,0
248115,0,3,6
248117,0,0,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248074,0,5,37
1,248085,0,0,1
2,248097,0,1,0
3,248115,0,3,6
4,248117,0,0,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248074,,5,37
1,248085,,,1
2,248097,,1,
3,248115,,3,6
4,248117,,,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 7 2022 11:34AM,248172,18,"Emersa Waterbox, LLC"
1,Oct 7 2022 11:27AM,248171,10,Methapharm-G
2,Oct 7 2022 11:26AM,248169,10,"Citieffe, Inc."
3,Oct 7 2022 11:26AM,248170,10,Methapharm-G
4,Oct 7 2022 11:24AM,248167,10,Methapharm-G
5,Oct 7 2022 11:20AM,248166,10,"Nextsource Biotechnology, LLC"
6,Oct 7 2022 10:58AM,248163,10,"Nextsource Biotechnology, LLC"
7,Oct 7 2022 10:53AM,248161,10,Methapharm-G
10,Oct 7 2022 10:48AM,248162,19,"Graystone, LLC"
16,Oct 7 2022 10:14AM,248150,15,"Virtus Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 7 2022 11:34AM,248172,18,"Emersa Waterbox, LLC",,,1
1,Oct 7 2022 11:27AM,248171,10,Methapharm-G,,,1
2,Oct 7 2022 11:26AM,248169,10,"Citieffe, Inc.",,,1
3,Oct 7 2022 11:26AM,248170,10,Methapharm-G,,,1
4,Oct 7 2022 11:24AM,248167,10,Methapharm-G,,,1
5,Oct 7 2022 11:20AM,248166,10,"Nextsource Biotechnology, LLC",,,1
6,Oct 7 2022 10:58AM,248163,10,"Nextsource Biotechnology, LLC",,,1
7,Oct 7 2022 10:53AM,248161,10,Methapharm-G,,,3
8,Oct 7 2022 10:48AM,248162,19,"Graystone, LLC",,,6
9,Oct 7 2022 10:14AM,248150,15,"Virtus Pharmaceuticals, LLC",,,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 7 2022 11:34AM,248172,18,"Emersa Waterbox, LLC",1,,
1,Oct 7 2022 11:27AM,248171,10,Methapharm-G,1,,
2,Oct 7 2022 11:26AM,248169,10,"Citieffe, Inc.",1,,
3,Oct 7 2022 11:26AM,248170,10,Methapharm-G,1,,
4,Oct 7 2022 11:24AM,248167,10,Methapharm-G,1,,
5,Oct 7 2022 11:20AM,248166,10,"Nextsource Biotechnology, LLC",1,,
6,Oct 7 2022 10:58AM,248163,10,"Nextsource Biotechnology, LLC",1,,
7,Oct 7 2022 10:53AM,248161,10,Methapharm-G,3,,
8,Oct 7 2022 10:48AM,248162,19,"Graystone, LLC",6,,
9,Oct 7 2022 10:14AM,248150,15,"Virtus Pharmaceuticals, LLC",4,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 7 2022 11:34AM,248172,18,"Emersa Waterbox, LLC",1,,
1,Oct 7 2022 11:27AM,248171,10,Methapharm-G,1,,
2,Oct 7 2022 11:26AM,248169,10,"Citieffe, Inc.",1,,
3,Oct 7 2022 11:26AM,248170,10,Methapharm-G,1,,
4,Oct 7 2022 11:24AM,248167,10,Methapharm-G,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 7 2022 11:34AM,248172,18,"Emersa Waterbox, LLC",1.0,NaN,NaN
1,Oct 7 2022 11:27AM,248171,10,Methapharm-G,1.0,NaN,NaN
2,Oct 7 2022 11:26AM,248169,10,"Citieffe, Inc.",1.0,NaN,NaN
3,Oct 7 2022 11:26AM,248170,10,Methapharm-G,1.0,NaN,NaN
4,Oct 7 2022 11:24AM,248167,10,Methapharm-G,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 7 2022 11:34AM,248172,18,"Emersa Waterbox, LLC",1.0,0.0,0.0
1,Oct 7 2022 11:27AM,248171,10,Methapharm-G,1.0,0.0,0.0
2,Oct 7 2022 11:26AM,248169,10,"Citieffe, Inc.",1.0,0.0,0.0
3,Oct 7 2022 11:26AM,248170,10,Methapharm-G,1.0,0.0,0.0
4,Oct 7 2022 11:24AM,248167,10,Methapharm-G,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2729717,18.0,9.0,10.0
15,992521,64.0,18.0,0.0
16,2729328,42.0,5.0,0.0
18,248172,1.0,0.0,0.0
19,248162,6.0,0.0,0.0
20,248085,1.0,0.0,0.0
21,496253,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2729717,18.0,9.0,10.0
1,15,992521,64.0,18.0,0.0
2,16,2729328,42.0,5.0,0.0
3,18,248172,1.0,0.0,0.0
4,19,248162,6.0,0.0,0.0
5,20,248085,1.0,0.0,0.0
6,21,496253,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,18.0,9.0,10.0
1,15,64.0,18.0,0.0
2,16,42.0,5.0,0.0
3,18,1.0,0.0,0.0
4,19,6.0,0.0,0.0
5,20,1.0,0.0,0.0
6,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,18.0
1,15,Released,64.0
2,16,Released,42.0
3,18,Released,1.0
4,19,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21
Status,,,,,,,
Completed,10.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,9.0,18.0,5.0,0.0,0.0,0.0,1.0
Released,18.0,64.0,42.0,1.0,6.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21
0,Completed,10.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,9.0,18.0,5.0,0.0,0.0,0.0,1.0
2,Released,18.0,64.0,42.0,1.0,6.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21
0,Completed,10.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,9.0,18.0,5.0,0.0,0.0,0.0,1.0
2,Released,18.0,64.0,42.0,1.0,6.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()